In [1]:
#Dependencies
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import json
import time
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from config import consumer_key, consumer_secret, access_token, access_token_secret
analyzer=SentimentIntensityAnalyzer()

In [2]:
#Auth for twitter
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
#Add a tweet mentioning the plotbot(myself) to analyze the CNN data
#api.update_status("@Haidy40802912 Analyze: @CNN")
targets_list=[]

In [3]:
#Define a Function to Retrive the latest 500 tweets for the target user and post them to
#(Plotbot "considered it Drishti_Y_UCI" cause Plotbot doesn't follow me) 
def iterate_tweets_target_user(target_user_to_be_defined,target_ordered_by, mentioned_bot_to_define):
    target_user = "@" +target_user_to_be_defined
    date=[]
    tweets_ago=[]
    results_compound=[]
    oldest_tweet=None
    counter=-1

    for x in range (1,6):
        public_tweets = api.user_timeline(target_user, count=100, max_id=oldest_tweet)
        for tweet in public_tweets:
            date.append(tweet["created_at"])
            tweets_ago.append(counter)
            results = analyzer.polarity_scores(tweet["text"])
            results_compound.append(results["compound"])
            oldest_tweet = tweet["id"]-1
            counter-=1
    Sentiment_Analysis = pd.DataFrame({"Date":date,
                                   "Tweets_ago":tweets_ago,
                                   "Tweets_Polarity":results_compound})
    todays_date= (datetime.datetime.now()).strftime("%m/%d/%Y")
    x_min= Sentiment_Analysis["Tweets_ago"].min()
    x_max=Sentiment_Analysis["Tweets_ago"].max()+1
    fig1, ax1=plt.subplots()
    ax1.plot(Sentiment_Analysis["Tweets_ago"],Sentiment_Analysis["Tweets_Polarity"],alpha=0.8,marker="o",linewidth=0.5,color="steelblue")
    ax1.grid(True, color='white')
    ax1.set_facecolor('lightgrey')
    plt.xlabel("Tweets Ago")
    plt.ylabel("Tweets Polarity")
    plt.title(f"Sentiment Analysis of Tweets ({todays_date})")
    plt.xlim(x_min,x_max)
    plt.legend(fontsize=5, mode="Expanded",labelspacing=0.5, bbox_to_anchor=(1,0.7), labels=["Tweets" + "\n"+f"@{target_ordered_by}"])
    plt.show()
    plt.savefig("SentimentAnalysisofTweets.png")
    api.update_with_media("SentimentAnalysisofTweets.png",f"Hi @{mentioned_bot} ; Kindly find below the sentiment analysis for @{latest_mention_tweets_target}")

In [7]:
#Check information in my account for mentions in a specific format of (" Analyze: @CNN") I used "Drishti's account" as PlotBot
#Since Plotbot is not following me yet
mention_tweets = api.user_timeline("@Haidy40802912")
string_to_look_for =  "Analyze:"

#while(True):
for tweet in mention_tweets:
    if string_to_look_for in tweet["text"]:
        latest_mention_tweets_target = mention_tweets[0]["entities"]["user_mentions"][-1]["screen_name"]
        if latest_mention_tweets_target not in targets_list:
            #not to re-analyze the targets already analyzed
            targets_list.append(latest_mention_tweets_target)
            target_ordered_from = mention_tweets[0]["user"]["screen_name"]
            mentioned_bot = mention_tweets[0]["entities"]["user_mentions"][0]["screen_name"]
            iterate_tweets_target_user(latest_mention_tweets_target,target_ordered_from,mentioned_bot)
            break
        else:
            print("Already Analyzed this Target Subject before")
    else: 
        skip=True
#time.sleep(300)

Already Analyzed this Target Subject before
